# Binary Quadratic Model: Qubo & Ising 

Any qlassf function can be transformed to a binary quadratic model using the `to_bqm` function. In this example we want factorize the number 15.

In [9]:
from qlasskit import qlassf, Qint4, Qint3

@qlassf
def test_factor(a: Qint3, b: Qint3) -> Qint4:
    return Qint4(15) - (a * b)

bqm = test_factor.to_bqm()

print("Vars:", bqm.num_variables, "\nInteractions:", bqm.num_interactions)

Vars: 14 
Interactions: 41


A qlassf function can also be exported as a QUBO model using `to_bqm('qubo')` or as an Ising model using `to_bqm('ising')`.


### Running on simulated sampler annealer

Now we can run a simulated sampler annealer to minimize this function; qlasskit offer a `decode_samples` helper function that translates sample result to the high level types of qlasskit.

In [10]:
import neal
from qlasskit.bqm import decode_samples

sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10)
decoded_samples = decode_samples(test_qubo, sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.sample)

{'a': 5, 'b': 3}


The result is 5 and 3 as expected (since 5 times 3 is equal to 15).

### Running on DWave annealer